In [53]:
'''
Author: Adam Warren
Date: 24/10/2023

Current Limitations:
- This only works where the names of the cresta zones are given
- e.g. there is no automapping of the cresta zone name -> number

Tool Capabilities:
  1. Checks the cresta zone as labelled in the raw data for:
    1. if the cresta zone and lat long are consistent
      1. This shows the user where they are correct, or if not, reports to the user 
         where the cresta is according to the given lat long.
    2. if the lat long isn't within a cresta zone
       in the workflow, using this after ankits tool means that these should 
       in theory only occur where the resolution is low
       
Outputs:
  1. User reporting .txt file that covers the key statistics 
  2. .html file that maps the inputs, with each latlong/cresta relationship shown
  3. Excel document outputs, seperated into the 3 categories, where necessary  
'''

from rtree.index import Index
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point 
import folium
from folium.plugins import MarkerCluster
import tkinter
from tkinter import filedialog

'''ORGANISE INPUTS'''

data = None
gdf = None
dir_path = None
region_col = None

def get_excel_doc():
    global data, dir_path, region_col
    root = tkinter.Tk()
    excel_doc = filedialog.askopenfilename()
    root.withdraw()
    dir_path = excel_doc.replace(excel_doc.split("/")[-1], "")
    if excel_doc.split(".")[-1] in ["xlsx", "xls"]:
        data = pd.read_excel(excel_doc)
        # standardise column names 
        lat_col = input("Input the name of your latitude column here:")
        long_col = input("Input the name of your longitude column here:")
        region_col = input("Input the name of your cresta zone column here:")
        locnum_col = input("Input the name of your location number column here:")
        data = data.rename(columns={lat_col: '_LAT_', long_col: '_LONG_', region_col: '_REGION_', locnum_col: '_LOCNUM_'})
    else:
        print("you chose the wrong file type, try again.")
get_excel_doc()

def get_gdf():
    global gdf
    root = tkinter.Tk()
    my_file_path = filedialog.askopenfilename()
    root.withdraw()
    if my_file_path.split(".")[-1] in ["shp"]:
        gdf = gpd.read_file(my_file_path)
    else:
        print("you chose the wrong file type, try again.")
get_gdf()

'''CREATE A FUNCTION TO PLOT THE MAPS'''

def create_folium(inside_df, inside_points_dict, ambiguity_points_dict, outside_points_dict, file_path_to_save):
    # Create a Folium map centered around the mean
    m = folium.Map(location=[inside_df['_LAT_'].mean(), inside_df['_LONG_'].mean()], zoom_start=4)
    
    # Add markers for points with cresta ambiguity
    cresta_ambiguity_cluster = MarkerCluster(name="Cresta/Coordinate Inconsistency", options={'disableClusteringAtZoom':10},show=True).add_to(m)
    for (lat_long, polygon_stated), info in ambiguity_points_dict.items():
        count = info['count']
        polygon_found = info.get('polygon_found', 'Not Found')
        folium.Marker(lat_long, 
                      popup=f'''<h5 style="width: 120px;">Ambiguity</h5><p style="width: 120px;">Cresta Stated: {polygon_stated}<br> Cresta Found: {polygon_found}<br>Count: {count}<br>Latitude: {lat_long[0]}<br>Longitude: {lat_long[1]}</p>''',
                      max_width=1500, icon=folium.Icon(color='orange')).add_to(cresta_ambiguity_cluster)
    
    # Add markers for points inside the cresta
    inside_cluster = MarkerCluster(name="Correctly Geocoded Coordinate Points", options={'disableClusteringAtZoom':10},show=False).add_to(m)
    for (lat_long, polygon), info in inside_points_dict.items():
        count = info['count']
        folium.Marker(lat_long, popup=f'''<h5 style="width: 120px;">Inside</h5><p style="width: 120px;">Cresta: {polygon}<br>Count: {count}<br>Latitude: {lat_long[0]}<br>Longitude: {lat_long[1]}</p>''',
                      max_width=1500, icon=folium.Icon(color='green')).add_to(inside_cluster)

    # Add markers for points outside the cresta
    outside_cluster = MarkerCluster(name="Outside Cresta Zones", options={'disableClusteringAtZoom':10},show=True).add_to(m)
    for (lat_long, polygon_stated), info in outside_points_dict.items():
        count = info['count']
        folium.Marker(lat_long, 
                      popup=f'''<h5 style="width: 120px;">Outside</h5><p style="width: 120px;">Cresta Stated: {polygon_stated}<br>Count: {count}<br>Latitude: {lat_long[0]}<br>Longitude: {lat_long[1]}</p>''',
                      max_width=1500, icon=folium.Icon(color='red')).add_to(outside_cluster)

    # Add the crestas on to the map as individual zones and as a whole
    all_crestas = folium.FeatureGroup(name='All Crestas', show=True)
    for idx, row in country_gdf.iterrows():
        folium.GeoJson(row['geometry']).add_to(all_crestas)
    all_crestas.add_to(m)   
    for idx, row in country_gdf.iterrows():
        polygon_group = folium.FeatureGroup(name=row['CRESTA_DES'], show=False)
        folium.GeoJson(row['geometry']).add_to(polygon_group)
        polygon_group.add_to(m)

    # Adding various tiles option incorporated within Folium library
    folium.TileLayer('CartoDB positron').add_to(m) #Selected as Default
    folium.TileLayer('OpenStreetMap').add_to(m) 
    folium.TileLayer('Stamen Terrain').add_to(m)
    folium.TileLayer('Stamen Toner').add_to(m)
    folium.TileLayer('esrinatgeoworldmap', name='Esri NatGeo WorldMap', attr=' Esri, Delorme, NAVTEQ').add_to(m)
    
    # Layer control for each layers added can be checked & unchecked while selecting
    folium.LayerControl(position='topright',collapsed=True, autoZIndex=True).add_to(m)

    # add a legend 
    legend_html = """
            <div style="position: fixed; bottom:30px; left:30px; z-index:9999; font-size:11px;">
                <p><i class="fa fa-circle fa-1x" style="color:green"></i> Correct Cresta </p>
                <p><i class="fa fa-circle fa-1x" style="color:orange"></i> Cresta/Coordinate Inconsistency </p>
                <p><i class="fa fa-circle fa-1x" style="color:red"></i> Outside All Cresta Zones </p>
            </div>
            """
    m.get_root().html.add_child(folium.Element(legend_html))

    # Full Screen option
    folium.plugins.Fullscreen().add_to(m)
    
    # get map output
    m.save(file_path_to_save) 

''' SPATIAL ANALYSIS '''
    
def CamelCase(word):
    words = word.split() 
    camel_case = ' '.join(word.capitalize() for word in words)
    return camel_case


# user will define the country such that the gdf only contains relevant information.
country_gdf = gdf.loc[gdf['COUNTRY'] == 'Peru']

# start a spatial index
my_spatial_index = Index()
for id, country in country_gdf.iterrows():
    my_spatial_index.insert(id, country.geometry.bounds)

# initialise variables
inside = {}
inside_id = []
cresta_ambiguity = {}
cresta_ambiguity_id = []
outside = {}
outside_id = []

# check each row of data
for idx, row in data.iterrows():
    point = Point(row['_LONG_'], row['_LAT_']) 
    lat_long = (row['_LAT_'], row['_LONG_']) 
    # find the cresta the client has given, standardize input
    client_cresta = CamelCase(row['_REGION_'])   
    # get the polygon for given cresta
    cresta_geometry = (country_gdf.loc[country_gdf['CRESTA_DES'] == client_cresta, 'geometry']).values[0]
    
    if point.within(cresta_geometry):
        key = (lat_long, client_cresta)
        inside[key] = inside.get(key, {'polygon': client_cresta, 'count': 0})
        inside[key]['count'] += 1
        inside_id.append(row['_LOCNUM_'])
    
    else: 
        possible_matches_id = list(my_spatial_index.intersection(point.bounds))
        if possible_matches_id:
            for row_id in possible_matches_id:
                if country_gdf.geometry[row_id].contains(point):
                    key = (lat_long, client_cresta)
                    cresta_ambiguity[key] = cresta_ambiguity.get(key, {'polygon_stated': client_cresta, 'polygon_found': country_gdf['CRESTA_DES'][row_id], 'count': 0})
                    cresta_ambiguity[key]['count'] += 1
                    cresta_ambiguity_id.append(row['_LOCNUM_'])
                else: 
                    continue      

        else:
            key = (lat_long, client_cresta)
            outside[key] = outside.get(key, {'polygon_stated': client_cresta, 'count': 0})
            outside[key]['count'] += 1
            outside_id.append(row['_LOCNUM_'])
            
'''CREATE OUTPUTS '''

# create  outputs
inside_df = data[data['_LOCNUM_'].isin(inside_id)]

cresta_ambiguity_df = data[data['_LOCNUM_'].isin(cresta_ambiguity_id)]
for id, row in cresta_ambiguity_df.iterrows():
    global region_col
    col_name = region_col + '_found'
    lat_long = (row['_LAT_'], row['_LONG_'])
    client_cresta = CamelCase(row['_REGION_'])
    cresta_ambiguity_df.at[id, col_name] = cresta_ambiguity[(lat_long, client_cresta)]['polygon_found']
    
outside_df = data[data['_LOCNUM_'].isin(outside_id)]

map_path = dir_path + r'/Map Output.html'
create_folium(inside_df, inside, cresta_ambiguity, outside, map_path)

user_reporting_file_path = dir_path + (r'/User Reporting.txt')
percentage_ad = len(cresta_ambiguity_df)*100/len(data)
percentage_ocz = len(outside_df)*100/len(data)
file = open(user_reporting_file_path, "w")
file.write(f'Total correct cresta data points: {len(inside_id)} \n'
            f'Total data points with inconsistent cresta/latlongs: {len(cresta_ambiguity_df)} \n'
            f'Total data points outside any cresta zone: {len(outside_id)} \n'
            f'Total points: {len(data)} \n'
            f'Percentage of inconsistent data: {percentage_ad:.1f}% \n'
            f'Percentage of data outside any cresta zone: {percentage_ocz:.1f}% \n' )
file.close()

if len(inside_df) > 0:
    inside_df_path = dir_path + (r'/Correct Cresta Zone and Lat Longs.xlsx')
    inside_df.to_excel(inside_df_path, index=False)
if len(cresta_ambiguity_df) > 0:
    cresta_ambiguity_df_path = dir_path + (r'/Inconsistent Cresta Zone and Lat Longs.xlsx')
    cresta_ambiguity_df.to_excel(cresta_ambiguity_df_path, index=False)
if len(outside_df) > 0:
    outside_df_path = dir_path + (r'/Outside Any Cresta Zone Lat Longs.xlsx')
    outside_df.to_excel(outside_df_path, index=False)



Input the name of your latitude column here:LAT
Input the name of your longitude column here:LONG
Input the name of your cresta zone column here:REGION
Input the name of your location number column here:LOCNUM


C:\Users\adwarren\AppData\Local\Temp\1\ipykernel_9564\2301535815.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cresta_ambiguity_df.at[id, col_name] = cresta_ambiguity[(lat_long, client_cresta)]['polygon_found']
